<a href="https://colab.research.google.com/github/sumit1729/machineLearning/blob/main/cnn_rnn_lstm_gru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
!pip install -U bnlp_toolkit
import seaborn as sns
import gensim, re
from matplotlib import style

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 23.9 MB 86.8 MB/s 
     |████████████████████████████████| 1.2 MB 59.8 MB/s 
     |████████████████████████████████| 965 kB 57.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
data_test = pd.read_csv("Test.csv")
data_test.head()

data_test.shape
print(data_test)
data_test['Label'].value_counts()
data_train = pd.read_csv("Train.csv")
data_train.head()

FileNotFoundError: ignored

In [ ]:
data_train = pd.read_csv("Train.csv")
data_test = pd.read_csv("Test.csv")
data_val = pd.read_csv("Val.csv")
data_train = data_train.append(data_test)
data_train = data_train.append(data_val)
data_train.isnull().sum()

ax = sns.countplot(x='Label', data=data_train)

Target = data_train['Label']

Text=data_train['Data'].tolist()
text = [re.sub(r'[^\u0980-\u09FF ]+', '', sentence) for sentence in Text]

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from bnlp import BasicTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

features = 1000
tokenizer = Tokenizer(num_words = features)

tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index

X = tokenizer.texts_to_sequences(text)
#print(X)
X= pad_sequences(X)
#print(X)
print(X.shape)
y = np.asarray(pd.get_dummies(data_train.Label))
#y = np.asarray(data_train.Label)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, shuffle=False)



In [ ]:
print(X)

In [ ]:
from gensim.models import Word2Vec
from gensim.models import FastText
from bnlp import BengaliWord2Vec
import multiprocessing


tokenized_lines = []
for single_line in text:
  tokenized_lines.append(single_line.split())

#word_model = Word2Vec(tokenized_lines, vector_size=300,window=20, min_count=10,negative=1,workers=multiprocessing.cpu_count()-1, epochs=100)
fasttext_model = FastText(tokenized_lines, vector_size=30, window=20, min_count=10,negative=1,workers=multiprocessing.cpu_count()-1, epochs=20)

## Dataset
There are 15728 instances in the dataset. 40.3% of these instances are positive, 36.3% are negative and 22.9% are neutral. Average length of these sentences are 15.37±9.93 words. Out of almost 31k sentences, 15K sentences were selected in the final dataset. These comments were collected from prothom alo newspaper in Bangladesh and YouTube videos on 13 different topics. 

## fastText

fastText is a library for word representation and sentence classification. it has pre-trained word vectors for 157 languages, including Bangla. we first tokenize the text data and generate vectors from 500 features on a vector size of 30. we train the fastText model for 100 epochs. we check whether the model is working by checking most similar words found by the fastText model. 

fasttext_model.wv.most_similar("খাবার", topn=10)

[('খাবারই', 0.9291799664497375),
 ('খাবারটা', 0.8485726118087769),
 ('খাবারে', 0.8299297094345093),
 ('খাবারের', 0.7983461618423462),
 ('খাবারগুলো', 0.762176513671875),
 ('খাব', 0.7132189273834229),
 ('রান্নার', 0.7064151763916016),
 ('রান্না', 0.6435276865959167),
 ('জঘন্য', 0.6362612843513489),
 ('সুস্বাদু', 0.6212347745895386)]

In [ ]:
fasttext_model.wv.most_similar("খাবার", topn=20)

In [ ]:
embedding_matrix = np.zeros((len(fasttext_model.wv)+1, 30))

for i, vec in enumerate(fasttext_model.wv.vectors):
  embedding_matrix[i] = vec

# Embedding matrix

we then generate an embedding matrix from the fastText model word vectors. we will use these vectors on an embedding layer and then use rnn and cnn models under the embedding layer.

## Callbacks

we define a callback that will stop the execution if the model starts to overfit after 5 epochs if the difference is equal to 0.0001 and restore previous best weights.

## lstm layer

out dataset size is comparatively small and therefore if we put a large number of units, the model will start to overfit very quickly. therefore we kept the number of units at 8 and define the model.


# GRU(Gated Recurrent Unit)

# Dropout




---



In [ ]:
import tensorflow as tf
keras = tf.keras
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Embedding, CuDNNLSTM, Bidirectional, Dense
from keras import layers, callbacks
from keras.layers import Input
from keras.layers.convolutional import Conv1D
from keras.layers import LSTM
from keras.layers.convolutional import MaxPooling1D

early_stopping = callbacks.EarlyStopping(
    min_delta = 0.0001,
    patience = 20,
    restore_best_weights = True
)

model = keras.Sequential([
                          layers.Embedding(len(fasttext_model.wv) + 1, 128, input_length=X.shape[1]),
                          #layers.LSTM(64,dropout=0.3, return_sequences=True),
                          #layers.LSTM(16, dropout=0.3),
                          #layers.GRU(32, dropout=0.3, return_sequences=True),
                          #layers.GRU(16, dropout=0.3),
                          #layers.Conv1D(filters=16, kernel_size=4, activation="tanh"),
                          #layers.MaxPooling1D(),
                          #layers.Conv1D(filters=16, kernel_size=4, activation="relu"),
                          #layers.Flatten(),
                          #layers.GRU(128, dropout=0.3),
                          layers.Bidirectional(CuDNNLSTM(16, return_sequences=True)),
                          #layers.Dense(64, activation="relu"),
                          layers.Bidirectional(CuDNNLSTM(16, return_sequences=True)),
                          #layers.Bidirectional(CuDNNLSTM(16, return_sequences=False)),
                          #layers.Dense(32, activation="relu"),
                          #layers.Bidirectional(CuDNNLSTM(16, return_sequences=True)),
                          #layers.Bidirectional(CuDNNLSTM(16, return_sequences=True)),
                          #layers.Bidirectional(CuDNNLSTM(8, return_sequences=False)),
                          layers.Dense(8, activation="relu"),
                          layers.Dense(4, activation="relu"),
                          #layers.Flatten(),
                          #layers.Bidirectional(CuDNNLSTM(16)),
                          layers.Dense(3, activation="softmax")
])

model.summary()

In [ ]:
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
history= model.fit(
    X_train, y_train,
    validation_data=(X_test,y_test),
    batch_size=64,
    epochs=1000,
    #class_weight=class_weight,
    callbacks=[early_stopping],
    verbose=1
)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_pred=model.predict(X_test) 
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(y_test, axis=1)
cm = confusion_matrix(y_test, y_pred)
print(cm)

#disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=data_train['Label'])

#disp.plot()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.legend(['train', 'valid'])
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

In [ ]:
# scatter plot of blobs dataset
from sklearn.datasets import make_blobs
from matplotlib import pyplot

history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()
plt.xlabel('epoch')
plt.ylabel('loss')
print("Minimum Loss: {:0.4f}".format(history_df['loss'].min()))
print("Maximum accuracy: {:0.4f}".format(history_df['acc'].max()))
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()));
print("Maximum validation accuracy: {:0.4f}".format(history_df['val_acc'].max()));

In [ ]:
import matplotlib.pyplot as plt
plt.hist(y_train)
plt.show()

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_bool))

In [ ]:

import matplotlib.pyplot as plt
from mlxtend.evaluate import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test, y_pred_bool)
labels=['neutral','positive', 'negative']
sns.heatmap(cm, annot=True, fmt='d', linewidths=0.5, xticklabels=labels, yticklabels=labels)

In [ ]:
from mlxtend.evaluate import confusion_matrix



cm = confusion_matrix(y_target=y_test, 
                      y_predicted=y_pred_bool, 
                      binary=False)
cm

In [ ]:
cm = confusion_matrix(y_test, y_pred_bool)
import matplotlib.pyplot as plt
from mlxtend.evaluate import confusion_matrix
import seaborn as sns
labels=['neutral','positive', 'negative']
sns.heatmap(cm, annot=True, fmt='d', linewidths=0.5, xticklabels=labels, yticklabels=labels)